In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
from tqdm import tqdm
%pip install pandas_profiling
%pip install ipywidgets
#from ydata_profiling import ProfileReport


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
df_A=pd.read_csv('/Users/Corentin/Desktop/statapp/transactions_A.csv')
df_B=pd.read_csv('/Users/Corentin/Desktop/statapp/transactions_B.csv')

In [3]:
df_A.head()

,tx_id,sender_account_id,receiver_account_id,tx_amount,timestamp,tx_type,sender_bank_id,sender_first_name,sender_last_name,sender_country_code,sender_birth_date,sender_zip_code,receiver_bank_id,receiver_first_name,receiver_last_name,receiver_country_code,receiver_birth_date,receiver_zip_code,is_alert_solo,is_alert_pooled
0,3109,6822.0,NaN,229.97,2018-03-03,WIRE,A,Marie,Lee,FR,1937-08-23,27800,C,Maritn,Alvarez,FR,1930-08-29,89160,False,False
1,1829,NaN,9011.0,117.83,2018-03-03,WIRE,B,Kurt,Brignac,FR,1962-09-07,35660,A,Joseph,Gable,FR,1933-12-23,62116,False,False
2,1830,NaN,7140.0,117.83,2018-03-03,WIRE,B,Kurt,Brignac,FR,1962-09-07,35660,A,Shelley,Spencer,FR,1944-12-23,34360,False,False
3,1831,NaN,1822.0,526.02,2018-03-03,WIRE,C,Shelly,Jethva,CN,1961-06-29,65230,A,Ambrose,Cole,FR,1958-04-07,3800,False,False
4,1832,5390.0,NaN,35.79,2018-03-03,WIRE,A,Judy,James,FR,1942-04-08,42140,C,Jose,Fiume,FR,1981-08-28,66820,False,False


In [4]:
df_B['tx_type'].value_counts()

WIRE     381529
CHECK     76537
Name: tx_type, dtype: int64

## Base clients pooling (sans infos sur les tx)

In [5]:
df_A.columns

Index(['tx_id', 'sender_account_id', 'receiver_account_id', 'tx_amount',
       'timestamp', 'tx_type', 'sender_bank_id', 'sender_first_name',
       'sender_last_name', 'sender_country_code', 'sender_birth_date',
       'sender_zip_code', 'receiver_bank_id', 'receiver_first_name',
       'receiver_last_name', 'receiver_country_code', 'receiver_birth_date',
       'receiver_zip_code', 'is_alert_solo', 'is_alert_pooled'],
      dtype='object')

In [6]:
df_base_client1 = df_A[['sender_first_name','sender_last_name','sender_birth_date',
                       'sender_account_id', 'sender_bank_id','sender_country_code',
                       'sender_zip_code']].copy()

df_base_client2 = df_A[['receiver_first_name','receiver_last_name','receiver_birth_date', 
                        'receiver_account_id', 'receiver_bank_id','receiver_country_code',
                       'receiver_zip_code']].copy()

df_base_client1.head(3)

df_base_client1.drop_duplicates(keep = 'first', inplace=True)
df_base_client2.drop_duplicates(keep = 'first', inplace=True)

df_base_client1.rename(columns = {'sender_first_name': 'First_name', 'sender_last_name': 'Last_name',
                                 'sender_birth_date' : 'birth_date', ' sender_account_id' : 'account_id',
                                  'sender_bank_id': 'bank_id','sender_country_code': 'country_code',
                                  'sender_zip_code': 'zip_code'},
                       inplace=True)

df_base_client2.rename(columns = {'receiver_first_name': 'First_name', 'receiver_last_name': 'Last_name',
                                 'receiver_birth_date' : 'birth_date',' receiver_account_id' : 'account_id',
                                  'receiver_bank_id': 'bank_id','receiver_country_code': 'country_code',
                                  'receiver_zip_code': 'zip_code'},
                       inplace=True)

df_base_client1.sort_values(by=['First_name', 'Last_name'],inplace=True)
df_base_client2.sort_values(by=['First_name', 'Last_name'],inplace=True)

df_base_client_A= pd.concat([df_base_client1,df_base_client2])
df_base_client_A.drop_duplicates(keep = 'first', inplace=True)
df_base_client_A

,First_name,Last_name,birth_date,sender_account_id,bank_id,country_code,zip_code,receiver_account_id
17672,Aaron,Behrmann,1955-07-05,6540.0,A,FR,40160,NaN
8665,Aaron,Coatney,1957-11-29,NaN,B,FR,60240,NaN
887,Aaron,Davis,1947-09-15,NaN,C,FR,46240,NaN
4389,Aaron,Gaskin,1942-04-25,2260.0,A,FR,71390,NaN
36813,Aaron,Lykins,1964-11-06,1961.0,A,FR,77540,NaN
...,...,...,...,...,...,...,...,...
7530,Yvonne,Seip,1976-02-10,NaN,A,FR,80540,3513.0
917,Zachary,Kawai,1929-05-30,NaN,A,FR,63450,753.0
17319,Zonia,Bucci,1955-11-23,NaN,C,SL,61110,NaN
6164,Zora,King,1934-09-30,NaN,A,FR,67250,5412.0


In [7]:
df_base_client1 = df_B[['sender_first_name','sender_last_name','sender_birth_date',
                       'sender_account_id', 'sender_bank_id','sender_country_code',
                       'sender_zip_code']].copy()

df_base_client2 = df_B[['receiver_first_name','receiver_last_name','receiver_birth_date', 
                        'receiver_account_id', 'receiver_bank_id','receiver_country_code',
                       'receiver_zip_code']].copy()

df_base_client1.drop_duplicates(keep = 'first', inplace=True)
df_base_client2.drop_duplicates(keep = 'first', inplace=True)

df_base_client1.rename(columns = {'sender_first_name': 'First_name', 'sender_last_name': 'Last_name',
                                 'sender_birth_date' : 'birth_date', ' sender_account_id' : 'account_id',
                                  'sender_bank_id': 'bank_id','sender_country_code': 'country_code',
                                  'sender_zip_code': 'zip_code'},
                       inplace=True)

df_base_client2.rename(columns = {'receiver_first_name': 'First_name', 'receiver_last_name': 'Last_name',
                                 'receiver_birth_date' : 'birth_date',' receiver_account_id' : 'account_id',
                                  'receiver_bank_id': 'bank_id','receiver_country_code': 'country_code',
                                  'receiver_zip_code': 'zip_code'},
                       inplace=True)

df_base_client1.sort_values(by=['First_name', 'Last_name'],inplace=True)
df_base_client2.sort_values(by=['First_name', 'Last_name'],inplace=True)

df_base_client_B= pd.concat([df_base_client1,df_base_client2])
df_base_client_B.drop_duplicates(keep = 'first', inplace=True)
df_base_client_B

,First_name,Last_name,birth_date,sender_account_id,bank_id,country_code,zip_code,receiver_account_id
3564,Aaron,Coatney,1957-11-29,5655.0,B,FR,60240,NaN
27599,Aaron,Gaskin,1942-04-25,NaN,A,FR,71390,NaN
13641,Aaron,Grubb,1962-02-23,NaN,C,FR,77760,NaN
9443,Aaron,Martinez,1941-05-16,1504.0,B,FR,60130,NaN
22221,Aaron,Rash,1967-03-21,7204.0,B,FR,27930,NaN
...,...,...,...,...,...,...,...,...
588,Zachary,Haynes,1968-04-12,NaN,B,FR,56220,9294.0
3967,Zandra,Mays,1959-01-23,NaN,B,FR,51210,5045.0
712,Zoila,Guillory,1951-02-24,NaN,B,FR,46600,9565.0
13664,Zonia,Bucci,1955-11-23,NaN,C,SL,61110,NaN


# Objectif : éviter le data leakage donc on va prendre en compte que les variables du passé 

In [51]:
df_test= df_A[['sender_first_name','sender_last_name','sender_birth_date','timestamp']].copy()

df= pd.DataFrame(df_A.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).median())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'median_cum_sender_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
df_test

,sender_first_name,sender_last_name,sender_birth_date,timestamp,median_cum_sender_A
0,Aaron,Behrmann,1955-07-05,2018-04-07,586.61
1,Aaron,Behrmann,1955-07-05,2018-04-14,586.61
2,Aaron,Behrmann,1955-07-05,2018-04-21,586.61
3,Aaron,Behrmann,1955-07-05,2018-04-28,586.61
4,Aaron,Behrmann,1955-07-05,2018-05-05,586.61
...,...,...,...,...,...
837976,Zulma,Steffan,1945-02-10,2021-11-20,426.79
837977,Zulma,Steffan,1945-02-10,2021-12-04,426.79
837978,Zulma,Steffan,1945-02-10,2021-12-11,426.79
837979,Zulma,Steffan,1945-02-10,2021-12-18,426.79


In [52]:
df_test[(df_test.sender_first_name=='Abel')&(df_test.sender_last_name=='Eskew')]

,sender_first_name,sender_last_name,sender_birth_date,timestamp,median_cum_sender_A
656,Abel,Eskew,1927-05-29,2018-05-05,158.5
657,Abel,Eskew,1927-05-29,2018-05-05,158.5
658,Abel,Eskew,1927-05-29,2018-05-05,158.5
659,Abel,Eskew,1927-05-29,2018-05-05,158.5
660,Abel,Eskew,1927-05-29,2018-05-05,158.5
...,...,...,...,...,...
861,Abel,Eskew,1927-05-29,2021-12-25,158.5
862,Abel,Eskew,1927-05-29,2021-12-25,158.5
863,Abel,Eskew,1927-05-29,2021-12-25,158.5
864,Abel,Eskew,1927-05-29,2021-12-25,158.5


## On crée une base clients SENDER en ne prenant en compte que les tx passées

In [5]:
# On crée une base clients SENDER en ne prenant en compte que les tx passées 
#Pour cela, on va sort_values par sender 

df_test= df_A[['sender_first_name','sender_last_name','sender_birth_date','timestamp']].copy()
 
    
# ensuite, on groupby pour pouvoir faire des aggrégations 
# on choisit comme aggrégation : count, moyenne, mediane et variance

# On s'occupe du sender 
df= pd.DataFrame(df_A.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).count())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'count_cum_sender_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
   


df= pd.DataFrame(df_A.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).median())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'median_cum_sender_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)

    
df= pd.DataFrame(df_A.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).mean())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'mean_cum_sender_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)

    
df= pd.DataFrame(df_A.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).var())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'variance_cum_sender_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)


# On compte également le nombre de tx is_alert_solo
df= pd.DataFrame(df_A.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['is_alert_solo'].expanding(1).sum())
df =pd.DataFrame(df.reset_index()['is_alert_solo'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'is_alert_solo': 'nb_is_alert_solo_sender_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
df_BC_sender_A = df_test.copy()

### On fait la même chose pour le receiver

In [6]:
#On fait la même chose mais pour le receiver ! 

df_test= df_A[['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp']].copy()

df= pd.DataFrame(df_A.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['tx_amount'].expanding(1).count())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'count_cum_receiver_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
   


df= pd.DataFrame(df_A.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['tx_amount'].expanding(1).median())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'median_cum_receiver_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)

    
df= pd.DataFrame(df_A.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['tx_amount'].expanding(1).mean())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'mean_cum_receiver_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)

    
df= pd.DataFrame(df_A.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['tx_amount'].expanding(1).var())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'variance_cum_receiver_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)


# On compte également le nombre de tx is_alert_solo
df= pd.DataFrame(df_A.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['is_alert_solo'].expanding(1).sum())
df =pd.DataFrame(df.reset_index()['is_alert_solo'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'is_alert_solo': 'nb_is_alert_solo_receiver_A'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
df_BC_receiver_A = df_test.copy()

### On fait la même pour B 

In [7]:
df_test= df_B[['sender_first_name','sender_last_name','sender_birth_date','timestamp']].copy()
 
    
# ensuite, on groupby pour pouvoir faire des aggrégations 
# on choisit comme aggrégation : count, moyenne, mediane et variance

# On s'occupe du sender 
df= pd.DataFrame(df_B.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).count())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'count_cum_sender_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
   


df= pd.DataFrame(df_B.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).median())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'median_cum_sender_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)

    
df= pd.DataFrame(df_B.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).mean())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'mean_cum_sender_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)

    
df= pd.DataFrame(df_B.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['tx_amount'].expanding(1).var())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'variance_cum_sender_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)


# On compte également le nombre de tx is_alert_solo
df= pd.DataFrame(df_B.groupby(by=['sender_first_name','sender_last_name','sender_birth_date'], as_index=False)['is_alert_solo'].expanding(1).sum())
df =pd.DataFrame(df.reset_index()['is_alert_solo'])
df_test.sort_values(by=['sender_first_name','sender_last_name','sender_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'is_alert_solo': 'nb_is_alert_solo_sender_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
df_BC_sender_B = df_test.copy()

In [8]:
#On fait la même chose mais pour le receiver ! 

df_test= df_B[['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp']].copy()

df= pd.DataFrame(df_B.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['tx_amount'].expanding(1).count())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'count_cum_receiver_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
   


df= pd.DataFrame(df_B.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['tx_amount'].expanding(1).median())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'median_cum_receiver_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)

    
df= pd.DataFrame(df_B.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['tx_amount'].expanding(1).mean())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'mean_cum_receiver_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)

    
df= pd.DataFrame(df_B.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['tx_amount'].expanding(1).var())

df =pd.DataFrame(df.reset_index()['tx_amount'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'tx_amount': 'variance_cum_receiver_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)


# On compte également le nombre de tx is_alert_solo
df= pd.DataFrame(df_B.groupby(by=['receiver_first_name','receiver_last_name','receiver_birth_date'], as_index=False)['is_alert_solo'].expanding(1).sum())
df =pd.DataFrame(df.reset_index()['is_alert_solo'])
df_test.sort_values(by=['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'],inplace=True)
df_test.reset_index(inplace=True) 
df_test.drop(['index'], axis=1, inplace=True)
df.rename(columns={'is_alert_solo': 'nb_is_alert_solo_receiver_B'},inplace=True)
df_test= pd.concat([df_test,df],axis=1)
df_BC_receiver_B = df_test.copy()

In [9]:
df_BC_receiver_B

,receiver_first_name,receiver_last_name,receiver_birth_date,timestamp,count_cum_receiver_B,median_cum_receiver_B,mean_cum_receiver_B,variance_cum_receiver_B,nb_is_alert_solo_receiver_B
0,Aaron,Behrmann,1955-07-05,2018-04-28,1.0,2047.420,2.047420e+03,NaN,0.0
1,Aaron,Behrmann,1955-07-05,2018-12-01,2.0,7543.215,7.543215e+03,6.040753e+07,0.0
2,Aaron,Behrmann,1955-07-05,2020-03-14,3.0,13039.010,2.187752e+05,1.338871e+11,0.0
3,Aaron,Behrmann,1955-07-05,2021-02-27,4.0,327139.095,1.853784e+06,1.078228e+13,0.0
4,Aaron,Behrmann,1955-07-05,2021-12-18,5.0,641239.180,5.777995e+06,8.508385e+13,0.0
...,...,...,...,...,...,...,...,...,...
458061,Zulma,Steffan,1945-02-10,2021-02-06,17.0,154.820,3.908947e+05,2.595521e+12,0.0
458062,Zulma,Steffan,1945-02-10,2021-03-27,18.0,154.820,3.691870e+05,2.451325e+12,0.0
458063,Zulma,Steffan,1945-02-10,2021-06-05,19.0,154.820,3.497642e+05,2.322308e+12,0.0
458064,Zulma,Steffan,1945-02-10,2021-08-14,20.0,154.820,3.322838e+05,2.206193e+12,0.0




# Enrichissement du dataset (par l'intermédiaire de la base de données clients) 

Dans un premier temps, on va réaliser la modélisation avec le df_merge grâce aux tx_id. 
Ensuite on fera exactement la même chose mais avec le df_merge grâce au pooling. 


In [10]:
df_A=pd.read_csv('/Users/Corentin/Desktop/statapp/transactions_A.csv')
df_B=pd.read_csv('/Users/Corentin/Desktop/statapp/transactions_B.csv')

In [11]:
# On accède à toutes les tx_id présentes dans les deux df
L1 = list(df_A['tx_id'].values) 
L2 = list(df_B['tx_id'].values)

In [12]:
df_merge = pd.concat([df_A,df_B],axis=0)
df_merge.drop_duplicates(subset=['tx_id'], inplace=True)
df_merge.sort_values(by=['sender_first_name','sender_last_name','timestamp'], inplace=True)
df_merge.reset_index(drop=True, inplace=True)
df_merge

,tx_id,sender_account_id,receiver_account_id,tx_amount,timestamp,tx_type,sender_bank_id,sender_first_name,sender_last_name,sender_country_code,sender_birth_date,sender_zip_code,receiver_bank_id,receiver_first_name,receiver_last_name,receiver_country_code,receiver_birth_date,receiver_zip_code,is_alert_solo,is_alert_pooled
0,33859,6540.0,9883.0,586.61,2018-04-07,WIRE,A,Aaron,Behrmann,FR,1955-07-05,40160,A,Ross,Sanchez,FR,1918-07-18,17360,False,False
1,40948,6540.0,9883.0,586.61,2018-04-14,WIRE,A,Aaron,Behrmann,FR,1955-07-05,40160,A,Ross,Sanchez,FR,1918-07-18,17360,False,False
2,42015,6540.0,9883.0,586.61,2018-04-21,CHECK,A,Aaron,Behrmann,FR,1955-07-05,40160,A,Ross,Sanchez,FR,1918-07-18,17360,False,False
3,50884,6540.0,9883.0,586.61,2018-04-28,WIRE,A,Aaron,Behrmann,FR,1955-07-05,40160,A,Ross,Sanchez,FR,1918-07-18,17360,False,False
4,60406,6540.0,9883.0,586.61,2018-05-05,WIRE,A,Aaron,Behrmann,FR,1955-07-05,40160,A,Ross,Sanchez,FR,1918-07-18,17360,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083393,1286129,7592.0,NaN,426.79,2021-11-20,WIRE,A,Zulma,Steffan,FR,1945-02-10,66320,C,Emily,Cooper,FR,1962-12-11,61100,False,False
1083394,1302564,7592.0,NaN,426.79,2021-12-04,CHECK,A,Zulma,Steffan,FR,1945-02-10,66320,C,Emily,Cooper,FR,1962-12-11,61100,False,False
1083395,1306496,7592.0,NaN,426.79,2021-12-11,WIRE,A,Zulma,Steffan,FR,1945-02-10,66320,C,Emily,Cooper,FR,1962-12-11,61100,False,False
1083396,1315640,7592.0,NaN,426.79,2021-12-18,CHECK,A,Zulma,Steffan,FR,1945-02-10,66320,C,Emily,Cooper,FR,1962-12-11,61100,False,False


In [13]:
df_BC_sender_A.drop(['sender_first_name','sender_last_name','sender_birth_date','timestamp'], axis =1, inplace=True )
df_BC_sender_B.drop(['sender_first_name','sender_last_name','sender_birth_date','timestamp'], axis =1, inplace=True )
df_BC_receiver_A.drop(['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'], axis =1, inplace=True )
df_BC_receiver_B.drop(['receiver_first_name','receiver_last_name','receiver_birth_date','timestamp'], axis =1, inplace=True )

### SENDER A 

In [14]:
df_merge.reset_index(drop=True, inplace=True) 

masque= df_merge['tx_id'].isin(L1)

df_filtre = df_merge.loc[masque]

df_filtre_restant=df_merge.loc[~masque]

df_filtre.reset_index(inplace=True) 

df_filtre.drop(['index'], axis=1, inplace=True)

df_filtre= pd.concat([df_filtre,df_BC_sender_A],axis=1)
df_merge= pd.concat([df_filtre,df_filtre_restant],axis=0)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
df_merge[(df_merge.sender_first_name=='Abel')&(df_merge.sender_last_name=='Eskew')][['timestamp','sender_first_name','receiver_bank_id','sender_bank_id','sender_last_name','tx_amount','median_cum_sender_A','count_cum_sender_A']]

,timestamp,sender_first_name,receiver_bank_id,sender_bank_id,sender_last_name,tx_amount,median_cum_sender_A,count_cum_sender_A
656,2018-05-05,Abel,B,A,Eskew,158.5,158.5,1.0
657,2018-05-05,Abel,B,A,Eskew,158.5,158.5,2.0
658,2018-05-05,Abel,A,A,Eskew,158.5,158.5,3.0
659,2018-05-05,Abel,C,A,Eskew,158.5,158.5,4.0
660,2018-05-05,Abel,C,A,Eskew,158.5,158.5,5.0
...,...,...,...,...,...,...,...,...
861,2021-12-25,Abel,A,A,Eskew,158.5,158.5,206.0
862,2021-12-25,Abel,B,A,Eskew,158.5,158.5,207.0
863,2021-12-25,Abel,A,A,Eskew,158.5,158.5,208.0
864,2021-12-25,Abel,A,A,Eskew,158.5,158.5,209.0


### SENDER B 

In [16]:
df_merge.reset_index(drop=True, inplace=True) 

masque= df_merge['tx_id'].isin(L2)

df_filtre = df_merge.loc[masque]

df_filtre_restant=df_merge.loc[~masque]

df_filtre.reset_index(inplace=True) 

df_filtre.drop(['index'], axis=1, inplace=True)

df_filtre= pd.concat([df_filtre,df_BC_sender_B],axis=1)
df_merge= pd.concat([df_filtre,df_filtre_restant],axis=0)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### RECEIVER A

In [17]:
df_ert=df_merge.copy()
df_ert.sort_values(by=['receiver_first_name','receiver_last_name','timestamp'], inplace=True)
df_ert.reset_index(drop=True, inplace=True) 
df_ert

,tx_id,sender_account_id,receiver_account_id,tx_amount,timestamp,tx_type,sender_bank_id,sender_first_name,sender_last_name,sender_country_code,...,count_cum_sender_A,median_cum_sender_A,mean_cum_sender_A,variance_cum_sender_A,nb_is_alert_solo_sender_A,count_cum_sender_B,median_cum_sender_B,mean_cum_sender_B,variance_cum_sender_B,nb_is_alert_solo_sender_B
0,10053,2470.0,6540.0,417.05,2018-03-17,WIRE,A,Corinne,Miller,FR,...,1.0,417.05,417.050000,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,14515,NaN,6540.0,117.08,2018-03-17,WIRE,C,Todd,Eagan,FR,...,1.0,117.08,117.080000,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,20275,2470.0,6540.0,417.05,2018-03-24,CHECK,A,Corinne,Miller,FR,...,2.0,417.05,417.050000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
3,23937,2470.0,6540.0,417.05,2018-03-31,WIRE,A,Corinne,Miller,FR,...,3.0,417.05,417.050000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
4,28696,2470.0,6540.0,417.05,2018-04-07,WIRE,A,Corinne,Miller,FR,...,4.0,417.05,417.050000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083393,1257533,NaN,7592.0,154.82,2021-10-23,WIRE,B,Lance,Reeves,FR,...,92.0,154.82,154.820000,0.000000,0.0,115.0,404.92,404.92,0.0,3.0
1083394,1265738,NaN,7592.0,7.26,2021-10-30,WIRE,C,Emily,Cooper,FR,...,87.0,7.26,7.260000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
1083395,1271330,NaN,7592.0,165.17,2021-11-06,WIRE,C,John,Eshom,FR,...,57.0,165.17,165.170000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
1083396,1277812,NaN,7592.0,7.26,2021-11-13,WIRE,C,Emily,Cooper,FR,...,96.0,7.26,7.260000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN


In [18]:
masque= df_ert['tx_id'].isin(L1)


df_masque = df_ert.loc[masque]

df_restant= df_ert.loc[~masque]

df_masque.reset_index(drop=True, inplace=True) 

df_filtre= pd.concat([df_masque,df_BC_receiver_A],axis=1)

df_filtre.reset_index(drop=True, inplace=True) 
df_restant.reset_index(drop=True, inplace=True)

df_ert= pd.concat([df_filtre,df_restant], axis=0,  ignore_index=True)

### RECEIVER B

In [19]:
df_ert[['timestamp','receiver_first_name','receiver_bank_id','receiver_last_name','tx_amount','mean_cum_receiver_A','median_cum_receiver_A']]

,timestamp,receiver_first_name,receiver_bank_id,receiver_last_name,tx_amount,mean_cum_receiver_A,median_cum_receiver_A
0,2018-03-17,Aaron,A,Behrmann,417.05,417.0500,417.050
1,2018-03-17,Aaron,A,Behrmann,117.08,267.0650,267.065
2,2018-03-24,Aaron,A,Behrmann,417.05,317.0600,417.050
3,2018-03-31,Aaron,A,Behrmann,417.05,342.0575,417.050
4,2018-04-07,Aaron,A,Behrmann,417.05,357.0560,417.050
...,...,...,...,...,...,...,...
1083393,2021-02-27,Zonia,C,Bucci,160.25,NaN,NaN
1083394,2021-05-08,Zonia,C,Bucci,160.25,NaN,NaN
1083395,2021-07-17,Zonia,C,Bucci,160.25,NaN,NaN
1083396,2021-09-25,Zonia,C,Bucci,160.25,NaN,NaN


In [20]:
masque= df_ert['tx_id'].isin(L2)

df_masque = df_ert.loc[masque]
df_restant= df_ert.loc[~masque]

df_masque.reset_index(drop=True, inplace=True) 

df_filtre= pd.concat([df_masque,df_BC_receiver_B],axis=1)

df_filtre.reset_index(drop=True, inplace=True) 
df_restant.reset_index(drop=True, inplace=True) 
df_ert= pd.concat([df_filtre,df_restant], axis=0,  ignore_index=True)

In [21]:
df_merge= df_ert.copy()
df_merge.sort_values(by=['sender_first_name','sender_last_name','timestamp'], inplace=True)
df_merge.reset_index(drop=True, inplace=True) 
df_merge

,tx_id,sender_account_id,receiver_account_id,tx_amount,timestamp,tx_type,sender_bank_id,sender_first_name,sender_last_name,sender_country_code,...,count_cum_receiver_A,median_cum_receiver_A,mean_cum_receiver_A,variance_cum_receiver_A,nb_is_alert_solo_receiver_A,count_cum_receiver_B,median_cum_receiver_B,mean_cum_receiver_B,variance_cum_receiver_B,nb_is_alert_solo_receiver_B
0,33859,6540.0,9883.0,586.61,2018-04-07,WIRE,A,Aaron,Behrmann,FR,...,54.0,491.10,402.884815,3.804797e+04,0.0,NaN,NaN,NaN,NaN,NaN
1,40948,6540.0,9883.0,586.61,2018-04-14,WIRE,A,Aaron,Behrmann,FR,...,68.0,462.80,401.569853,3.731530e+04,0.0,NaN,NaN,NaN,NaN,NaN
2,42015,6540.0,9883.0,586.61,2018-04-21,CHECK,A,Aaron,Behrmann,FR,...,80.0,477.85,404.187875,3.711268e+04,0.0,NaN,NaN,NaN,NaN,NaN
3,50884,6540.0,9883.0,586.61,2018-04-28,WIRE,A,Aaron,Behrmann,FR,...,97.0,477.85,485.770722,2.943041e+05,0.0,NaN,NaN,NaN,NaN,NaN
4,60406,6540.0,9883.0,586.61,2018-05-05,WIRE,A,Aaron,Behrmann,FR,...,111.0,477.85,472.313784,2.645590e+05,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083393,1286129,7592.0,NaN,426.79,2021-11-20,WIRE,A,Zulma,Steffan,FR,...,242.0,426.79,4812.730000,4.862179e+09,1.0,NaN,NaN,NaN,NaN,NaN
1083394,1302564,7592.0,NaN,426.79,2021-12-04,CHECK,A,Zulma,Steffan,FR,...,243.0,426.79,4794.680864,4.842166e+09,1.0,NaN,NaN,NaN,NaN,NaN
1083395,1306496,7592.0,NaN,426.79,2021-12-11,WIRE,A,Zulma,Steffan,FR,...,244.0,426.79,4776.779672,4.822318e+09,1.0,NaN,NaN,NaN,NaN,NaN
1083396,1315640,7592.0,NaN,426.79,2021-12-18,CHECK,A,Zulma,Steffan,FR,...,245.0,426.79,4759.024612,4.802631e+09,1.0,NaN,NaN,NaN,NaN,NaN


In [27]:
df_merge[(df_merge.sender_first_name=='Abel')&(df_merge.sender_last_name=='Eskew')][['timestamp','sender_first_name','receiver_bank_id','sender_bank_id','sender_last_name','tx_amount','median_cum_sender_B','median_cum_sender_A','median_cum_sender_B','count_cum_sender_A']]

,timestamp,sender_first_name,receiver_bank_id,sender_bank_id,sender_last_name,tx_amount,median_cum_sender_B,median_cum_sender_A,median_cum_sender_B,count_cum_sender_A
1019,2018-05-05,Abel,B,A,Eskew,158.5,495.95,158.5,495.95,2.0
1020,2018-05-05,Abel,B,A,Eskew,158.5,495.95,158.5,495.95,1.0
1021,2018-05-05,Abel,A,A,Eskew,158.5,NaN,158.5,NaN,7.0
1022,2018-05-05,Abel,A,A,Eskew,158.5,NaN,158.5,NaN,10.0
1023,2018-05-05,Abel,C,A,Eskew,158.5,NaN,158.5,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...
1224,2021-12-25,Abel,C,A,Eskew,158.5,NaN,158.5,NaN,204.0
1225,2021-12-25,Abel,A,A,Eskew,158.5,NaN,158.5,NaN,203.0
1226,2021-12-25,Abel,C,A,Eskew,158.5,NaN,158.5,NaN,200.0
1227,2021-12-25,Abel,A,A,Eskew,158.5,NaN,158.5,NaN,209.0


## Question principale : que va apporter les informations de B à A ?

On analyse premièrement quelles informations supplémentaires obtient-on grâce aux 2 lignes. 
Pour cela, on récupère les tx_id présentes dans les 2 datasets et on regarde les 2 lignes correspondantes. 

In [40]:
# On accède à toutes les tx_id présentes dans les deux df
L1 = list(df_A['tx_id'].values) 
L2 = list(df_B['tx_id'].values)

L=np.isin(L1,L2).astype(int)*L1
L=L[L!= 0]  # liste des transactions appartenant aux 2 datasets 

In [35]:
def analyse_de_tx(nb_tx):
    "permet de comparer deux lignes d'une même transaction"
    display(df_A[df_A['tx_id']==nb_tx])
    display(df_B[df_B['tx_id']==nb_tx])

In [36]:
analyse_de_tx(L[145667])

,tx_id,sender_account_id,receiver_account_id,tx_amount,timestamp,tx_type,sender_bank_id,sender_first_name,sender_last_name,sender_country_code,sender_birth_date,sender_zip_code,receiver_bank_id,receiver_first_name,receiver_last_name,receiver_country_code,receiver_birth_date,receiver_zip_code,is_alert_solo,is_alert_pooled
582204,1319104,NaN,7202.0,152.67,2021-12-25,WIRE,B,Michael,Washington,FR,1943-07-27,74370,A,Natasha,Kuykendall,FR,1943-05-14,60530,False,False


,tx_id,sender_account_id,receiver_account_id,tx_amount,timestamp,tx_type,sender_bank_id,sender_first_name,sender_last_name,sender_country_code,sender_birth_date,sender_zip_code,receiver_bank_id,receiver_first_name,receiver_last_name,receiver_country_code,receiver_birth_date,receiver_zip_code,is_alert_solo,is_alert_pooled
457781,1319104,2604.0,NaN,152.67,2021-12-25,WIRE,B,Michael,Washington,FR,1943-07-27,74370,A,Natasha,Kuykendall,FR,1943-05-14,60530,False,False


Les 2 lignes sont équivalentes pour la plupart (totalité?) des transactions en commun. 
La seule différence est le sender_account_id ou le receiver_account_id qui n'ont pas réellement d'importance. 


#### ==> Une hypothèse que l'on peut formuler à ce stade est que la plus-value de merge les DF porte donc sur les clients ! 

## On va ajouter les informations clients (des deux bases de données A et B) au dataframe pour compléter notre dataset ! 

### On crée une métrique : fiabilité de l'individu qui vaut 1 si l'individu n'a jamais eu alert_solo

In [24]:
#Receiver
df_merge['reliability_receiver_A']= 1- df_merge['nb_is_alert_solo_receiver_A']/df_merge['count_cum_receiver_A']
df_merge['reliability_receiver_B']= 1- df_merge['nb_is_alert_solo_receiver_B']/df_merge['count_cum_receiver_B']

#Sender
df_merge['reliability_sender_A']= 1- df_merge['nb_is_alert_solo_sender_A']/df_merge['count_cum_sender_A']
df_merge['reliability_sender_B']= 1- df_merge['nb_is_alert_solo_sender_B']/df_merge['count_cum_sender_B']

### On met les dates en datetime 

In [25]:
df_merge['timestamp'] = pd.to_datetime(df_merge.timestamp, format='%Y-%m-%d', errors = 'ignore')
df_merge['sender_birth_date'] = pd.to_datetime(df_merge.sender_birth_date, format='%Y-%m-%d', errors = 'ignore')
df_merge['receiver_birth_date'] = pd.to_datetime(df_merge.receiver_birth_date, format='%Y-%m-%d', errors = 'ignore')

#### On crée également des nouvelles features : 
- écart en % de tx_amount par rapport à la médiane de tx_amount pour un client 
- jour de l'année/ jour du mois 

In [26]:
#Écart
df_merge['ecarts_tx_received_A']= (df_merge.tx_amount - df_merge.median_cum_receiver_A)/df_merge.median_cum_receiver_A*100
df_merge['ecarts_tx_received_B']= (df_merge.tx_amount - df_merge.median_cum_receiver_B)/df_merge.median_cum_receiver_B*100
df_merge['ecarts_tx_sent_A']= (df_merge.tx_amount - df_merge.median_cum_sender_A)/df_merge.median_cum_sender_A*100
df_merge['ecarts_tx_sent_B']= (df_merge.tx_amount - df_merge.median_cum_sender_B)/df_merge.median_cum_sender_B*100

#Jours de l'année/du mois 
df_merge['jour_année_tx']=df_merge['timestamp'].dt.dayofyear
df_merge['jour_mois_tx']= df_merge['timestamp'].dt.day

#Mois et année
df_merge['année_tx']=df_merge['timestamp'].dt.year
df_merge['mois_tx']=df_merge['timestamp'].dt.month

df_merge.head(5)

,tx_id,sender_account_id,receiver_account_id,tx_amount,timestamp,tx_type,sender_bank_id,sender_first_name,sender_last_name,sender_country_code,...,reliability_sender_A,reliability_sender_B,ecarts_tx_received_A,ecarts_tx_received_B,ecarts_tx_sent_A,ecarts_tx_sent_B,jour_année_tx,jour_mois_tx,année_tx,mois_tx
0,33859,6540.0,9883.0,586.61,2018-04-07,WIRE,A,Aaron,Behrmann,FR,...,1.0,NaN,19.448178,NaN,0.0,NaN,97,7,2018,4
1,40948,6540.0,9883.0,586.61,2018-04-14,WIRE,A,Aaron,Behrmann,FR,...,1.0,NaN,26.752377,NaN,0.0,NaN,104,14,2018,4
2,42015,6540.0,9883.0,586.61,2018-04-21,CHECK,A,Aaron,Behrmann,FR,...,1.0,NaN,22.760280,NaN,0.0,NaN,111,21,2018,4
3,50884,6540.0,9883.0,586.61,2018-04-28,WIRE,A,Aaron,Behrmann,FR,...,1.0,NaN,22.760280,NaN,0.0,NaN,118,28,2018,4
4,60406,6540.0,9883.0,586.61,2018-05-05,WIRE,A,Aaron,Behrmann,FR,...,1.0,NaN,22.760280,NaN,0.0,NaN,125,5,2018,5


In [30]:
df_merge.to_csv('/Users/Corentin/Desktop/statapp/df_merge_2.csv')

In [28]:
df_merge.columns

Index(['tx_id', 'sender_account_id', 'receiver_account_id', 'tx_amount',
       'timestamp', 'tx_type', 'sender_bank_id', 'sender_first_name',
       'sender_last_name', 'sender_country_code', 'sender_birth_date',
       'sender_zip_code', 'receiver_bank_id', 'receiver_first_name',
       'receiver_last_name', 'receiver_country_code', 'receiver_birth_date',
       'receiver_zip_code', 'is_alert_solo', 'is_alert_pooled',
       'count_cum_sender_A', 'median_cum_sender_A', 'mean_cum_sender_A',
       'variance_cum_sender_A', 'nb_is_alert_solo_sender_A',
       'count_cum_sender_B', 'median_cum_sender_B', 'mean_cum_sender_B',
       'variance_cum_sender_B', 'nb_is_alert_solo_sender_B',
       'count_cum_receiver_A', 'median_cum_receiver_A', 'mean_cum_receiver_A',
       'variance_cum_receiver_A', 'nb_is_alert_solo_receiver_A',
       'count_cum_receiver_B', 'median_cum_receiver_B', 'mean_cum_receiver_B',
       'variance_cum_receiver_B', 'nb_is_alert_solo_receiver_B',
       'reliability_